In [1]:
import pandas as pd
import numpy as np
import sklearn, xgboost
import warnings
warnings.filterwarnings('ignore')


x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")

In [2]:
# train과 test 모두 NA 값 X

In [3]:
x_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


In [4]:
# 삭제 : ID
# 더미 : Warehouse_block, Mode_of_Shipment, Product_importance, Genderm 
# 라벨인코딩 : Customer_care_calls
# 건들 ㄴㄴ : Customer_rating
# 숫자 : Cost_of_the_Product, Discount_offered, Weight_in_gms, Prior_purchases

In [5]:
COL_DEL = ['ID']
COL_DUM = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
COL_LE = ['Customer_care_calls']
COL_NUM = ['Cost_of_the_Product', 'Discount_offered', 'Weight_in_gms', 'Prior_purchases']
COL_NO = ['Customer_rating']

In [6]:
# 불필요 피쳐 삭제
X_train = x_train.drop(COL_DEL, axis = 1)
X_test = x_test.drop(COL_DEL, axis = 1)

In [7]:
# 원핫인코딩
X_train_dum = pd.get_dummies(X_train[COL_DUM])
X_test_dum = pd.get_dummies(X_test[COL_DUM])

In [8]:
X_train = pd.concat([X_train, X_train_dum], axis = 1)
X_test = pd.concat([X_test, X_test_dum], axis = 1)

In [9]:
X_train = X_train.drop(COL_DUM, axis = 1)
X_test = X_test.drop(COL_DUM, axis = 1)

In [10]:
# 라벨 인코딩

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(X_train[COL_LE])
X_train[COL_LE] = le.fit_transform(X_train[COL_LE])
X_test[COL_LE] = le.fit_transform(X_test[COL_LE])

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Customer_care_calls        6598 non-null   int64
 1   Customer_rating            6598 non-null   int64
 2   Cost_of_the_Product        6598 non-null   int64
 3   Prior_purchases            6598 non-null   int64
 4   Discount_offered           6598 non-null   int64
 5   Weight_in_gms              6598 non-null   int64
 6   Warehouse_block_A          6598 non-null   uint8
 7   Warehouse_block_B          6598 non-null   uint8
 8   Warehouse_block_C          6598 non-null   uint8
 9   Warehouse_block_D          6598 non-null   uint8
 10  Warehouse_block_F          6598 non-null   uint8
 11  Mode_of_Shipment_Flight    6598 non-null   uint8
 12  Mode_of_Shipment_Road      6598 non-null   uint8
 13  Mode_of_Shipment_Ship      6598 non-null   uint8
 14  Product_importance_high 

In [14]:
# split train - val data
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.3)

In [15]:
# Min-Max-scale or StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [131]:
# Min Max Scaler
#mms = MinMaxScaler()
#mms.fit(X_tr[COL_NUM])
#X_tr[COL_NUM] = mms.fit_transform(X_tr[COL_NUM])
#X_test[COL_NUM] = mms.fit_transform(X_test[COL_NUM])
#X_val[COL_NUM] = mms.fit_transform(X_val[COL_NUM])

In [16]:
# Standard Scaler
scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])

In [17]:
# Modeling

In [18]:
y_tr_target = y_tr.drop(['ID'], axis = 1)
y_val_target = y_val.drop(['ID'], axis = 1)

In [19]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(random_state = 11)
model_rf.fit(X_tr, y_tr_target.values.ravel())

RandomForestClassifier(random_state=11)

In [20]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(random_state = 22)
model_xgb.fit(X_tr, y_tr_target.values.ravel())

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [21]:
from sklearn.metrics import roc_auc_score

pred_rf = model_rf.predict_proba(X_val)[:,1]
pred_xgb = model_xgb.predict_proba(X_val)[:,1]

In [22]:
score_rf = roc_auc_score(y_val_target, pred_rf)
score_xgb = roc_auc_score(y_val_target, pred_xgb)

print(score_rf, score_xgb)

0.7348379707394094 0.7324816491416669


In [23]:
model_xgb = XGBClassifier()
model_xgb.fit(X_tr, y_tr_target.values.ravel())
pred_xgb = model_xgb.predict_proba(X_val)[:,1]
score_xgb = roc_auc_score(y_val_target, pred_xgb)
print(score_xgb)

0.7324816491416669


In [24]:
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv")
y_test_target = y_test.drop(['ID'], axis = 1)

In [25]:
xgb_pred = model_xgb.predict_proba(X_test)
rf_pred = model_rf.predict_proba(X_test)

In [26]:
roc_auc_score(y_test_target, xgb_pred[:,1])

0.7394497066174657

In [27]:
roc_auc_score(y_test_target, rf_pred[:,1])

0.7343670553404203